d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# 3.4 File Formats

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this notebook you:<br>
* Compare file formats and compression types
* Examine Parquet

In [0]:
%run ../Includes/Classroom-Setup

Data mounted to /mnt/davis ...

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Comparing File Formats<br>

Let's take a look at a colon delimited file sitting on S3.

In [0]:
%fs ls /mnt/davis/fire-calls/fire-calls-colon.txt

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-colon.txt,fire-calls-colon.txt,1829688314


Take a look at the first few lines of the file.

In [0]:
%fs head --maxBytes=1000 /mnt/davis/fire-calls/fire-calls-colon.txt

[Truncated to first 1000 bytes]
Call Number:Unit ID:Incident Number:Call Type:Call Date:Watch Date:Received DtTm:Entry DtTm:Dispatch DtTm:Response DtTm:On Scene DtTm:Transport DtTm:Hospital DtTm:Call Final Disposition:Available DtTm:Address:City:Zipcode of Incident:Battalion:Station Area:Box:Original Priority:Priority:Final Priority:ALS Unit:Call Type Group:Number of Alarms:Unit Type:Unit sequence in call dispatch:Fire Prevention District:Supervisor District:Neighborhooods - Analysis Boundaries:Location:RowID
001030101:E18:00306091:Medical Incident:04/12/2000:04/12/2000:"04/12/2000 09:00:29 PM":"04/12/2000 09:01:40 PM":"04/12/2000 09:02:00 PM":"":"":"":"":Other:"":2000 Block of 37TH AVE:SF:94116:B08:18:0757:3:3:3:false:"":1:ENGINE:1:8:4:Sunset/Parkside:(37.7487247711275, -122.495504020186):001030101-E18
001030104:M14:00030612:Medical Incident:04/12/2000:04/12/2000:"04/12/2000 09:09:02 PM":"04/12/2000 09:10:17 PM":"04/12/2000 09:10:29 PM":"04/12/2000 09:12:11 PM":"04/12/2000 09:19:36 PM":"04/12/2000 09:43:57 PM":"04/1

Create a temporary view of the file using `:` as the separator.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSV
USING CSV 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-colon.txt",
    header "true",
    sep ":"
  )

Take a look at the types of data in the table.

In [0]:
%sql
DESCRIBE fireCallsCSV

col_name,data_type,comment
Call Number,string,null
Unit ID,string,null
Incident Number,string,null
Call Type,string,null
Call Date,string,null
Watch Date,string,null
Received DtTm,string,null
Entry DtTm,string,null
Dispatch DtTm,string,null
Response DtTm,string,null


Are these data types correct? All of them are string types.

We need to tell Spark to infer the schema.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSV
USING CSV 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-colon.txt",
    header "true",
    sep ":",
    inferSchema "true"
  )

Now take a look at how Spark inferred the data types.

In [0]:
%sql
DESCRIBE fireCallsCSV

col_name,data_type,comment
Call Number,int,null
Unit ID,string,null
Incident Number,int,null
Call Type,string,null
Call Date,string,null
Watch Date,string,null
Received DtTm,string,null
Entry DtTm,string,null
Dispatch DtTm,string,null
Response DtTm,string,null


Wow, that took a long time just to figure out the schema for this file! 

Now let's try the same thing with compressed files (Gzip and Bzip formats).

Notice that the bzip file is the most compact - we will see if it is the fastest to operate on.

In [0]:
%fs ls /mnt/davis/fire-calls/fire-calls-colon.txt

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-colon.txt,fire-calls-colon.txt,1829688314


In [0]:
%fs ls /mnt/davis/fire-calls/fire-calls-1p.txt.gz

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.gz/_SUCCESS,_SUCCESS,0
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.gz/_committed_1074000646993103229,_committed_1074000646993103229,116
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.gz/_started_1074000646993103229,_started_1074000646993103229,0
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.gz/part-00000-tid-4222569572634380978-7f3cb922-b07c-4d71-b14f-f65d838c499a-15121-c000.csv.gz,part-00000-tid-4222569572634380978-7f3cb922-b07c-4d71-b14f-f65d838c499a-15121-c000.csv.gz,259566857


In [0]:
%fs ls /mnt/davis/fire-calls/fire-calls-1p.txt.bzip

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.bzip/_SUCCESS,_SUCCESS,0
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.bzip/_committed_7012599954241226118,_committed_7012599954241226118,117
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.bzip/_started_7012599954241226118,_started_7012599954241226118,0
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.bzip/part-00000-tid-7012599954241226118-f407e362-c5a4-49ec-973b-3420b6ae94fd-15136-c000.csv.bz2,part-00000-tid-7012599954241226118-f407e362-c5a4-49ec-973b-3420b6ae94fd-15136-c000.csv.bz2,192991035


Let's start by reading in the gzipped file.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSVgzip
USING CSV 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-1p.txt.gz",
    header "true",
    sep ":",
    inferSchema "true"
  )

Wow! That took way longer than inferring the schema on the uncompressed data. Even though it took up less storage space, we had to pay for that in computation.

You'll notice that the resulting view is comprised of only 1 partition, which makes this data very slow to query later on.

In [0]:
%python
sql("SELECT * FROM fireCallsCSVgzip").rdd.getNumPartitions()

Out[2]: 1

Let's compare the speed of reading in the gzip file to the bzip file!

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSVbzip
USING CSV 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-1p.txt.bzip",
    header "true",
    sep ":",
    inferSchema "true"
  )

In [0]:
%python
sql("SELECT * FROM fireCallsCSVbzip").rdd.getNumPartitions()

Out[3]: 8

Bzip is a "splittable" file format, so it is much better to use than gzip when working with row-based formats for querying later on.

Now let's go ahead and compare that to reading in from a columnar format: Parquet.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsParquet
USING Parquet 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-1p.parquet"
  )

In [0]:
%sql
DESCRIBE fireCallsParquet

col_name,data_type,comment
Call_Number,int,null
Unit_ID,string,null
Incident_Number,int,null
Call_Type,string,null
Call_Date,string,null
Watch_Date,string,null
Received_DtTm,string,null
Entry_DtTm,string,null
Dispatch_DtTm,string,null
Response_DtTm,string,null


Look at how fast it is to get the schema from a Parquet file! That is because the Parquet file stores the data and the associated metadata.

Compare the performance between the three file types. We are going to use a Python helper function called [timeit](https://ipython.org/ipython-doc/3/interactive/magics.html#magic-timeit) to calculate how long the query takes to execute.

In [0]:
%python
parquetDF = sql("SELECT * FROM fireCallsParquet")
%timeit -n1 -r1 parquetDF.select("City").where("City == 'San Francisco'").count()

6.65 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

In [0]:
%python
csvDF = sql("SELECT * FROM fireCallsCSV")
%timeit -n1 -r1 csvDF.select("City").where("City == 'San Francisco'").count()

35.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

In [0]:
%python
gzipDF = sql("SELECT * FROM fireCallsCSVgzip")
%timeit -n1 -r1 gzipDF.select("City").where("City == 'San Francisco'").count()

24.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

In [0]:
%python
bzipDF = sql("SELECT * FROM fireCallsCSVbzip")
%timeit -n1 -r1 bzipDF.select("City").where("City == 'San Francisco'").count()

1min 25s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

-sandbox
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Reading from Parquet Files

"Apache Parquet is a columnar storage format available to any project in the Hadoop ecosystem, regardless of the choice of data processing framework, data model or programming language."

<div style="text-align:right">
![parquet logo](https://parquet.apache.org/assets/img/parquet_logo.png)<br>
<a href="https://parquet.apache.org/" target="_blank">https&#58;//parquet.apache.org</a></div>

-sandbox
### About Parquet Files
* Free & Open Source.
* Increased query performance over row-based data stores.
* Provides efficient data compression.
* Designed for performance on large data sets.
* Supports limited schema evolution.
* Is a splittable "file format".
* A <a href="https://en.wikipedia.org/wiki/Column-oriented_DBMS" target="_blank">Column-Oriented</a> data store

&nbsp;&nbsp;&nbsp;&nbsp;** Row Format ** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Column Format**

<table style="border:0">

  <tr>
    <th>ID</th><th>Name</th><th>Score</th>
    <th style="border-top:0;border-bottom:0">&nbsp;</th>
    <th>ID:</th><td>1</td><td>2</td>
    <td style="border-right: 1px solid #DDDDDD">3</td>
  </tr>

  <tr>
    <td>1</td><td>john</td><td>4.1</td>
    <td style="border-top:0;border-bottom:0">&nbsp;</td>
    <th>Name:</th><td>john</td><td>mike</td>
    <td style="border-right: 1px solid #DDDDDD">sally</td>
  </tr>

  <tr>
    <td>2</td><td>mike</td><td>3.5</td>
    <td style="border-top:0;border-bottom:0">&nbsp;</td>
    <th style="border-bottom: 1px solid #DDDDDD">Score:</th>
    <td style="border-bottom: 1px solid #DDDDDD">4.1</td>
    <td style="border-bottom: 1px solid #DDDDDD">3.5</td>
    <td style="border-bottom: 1px solid #DDDDDD; border-right: 1px solid #DDDDDD">6.4</td>
  </tr>

  <tr>
    <td style="border-bottom: 1px solid #DDDDDD">3</td>
    <td style="border-bottom: 1px solid #DDDDDD">sally</td>
    <td style="border-bottom: 1px solid #DDDDDD; border-right: 1px solid #DDDDDD">6.4</td>
  </tr>

</table>

See also
* <a href="https://parquet.apache.org/" target="_blank">https&#58;//parquet.apache.org</a>
* <a href="https://en.wikipedia.org/wiki/Apache_Parquet" target="_blank">https&#58;//en.wikipedia.org/wiki/Apache_Parquet</a>

In [0]:
%sql
SELECT count(`Incident Number`)
FROM firecallsCSV
WHERE Priority > 1

count(Incident Number)
4401920


In [0]:
%sql
SELECT count(`Incident_Number`)
FROM firecallsParquet
WHERE Priority > 1

count(Incident_Number)
4401920


How much faster can this query get if we read from a partitioned Parquet file?

In [0]:
%fs ls /mnt/davis/fire-calls/fire-calls-8p.parquet

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/_SUCCESS,_SUCCESS,0
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/_committed_1695916237803481236,_committed_1695916237803481236,824
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/_committed_3872868080952213493,_committed_3872868080952213493,1634
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/_committed_vacuum5173795997807967562,_committed_vacuum5173795997807967562,96
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/_started_3872868080952213493,_started_3872868080952213493,0
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/part-00000-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22403-c000.snappy.parquet,part-00000-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22403-c000.snappy.parquet,70689109
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/part-00001-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22404-c000.snappy.parquet,part-00001-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22404-c000.snappy.parquet,70249066
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/part-00002-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22405-c000.snappy.parquet,part-00002-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22405-c000.snappy.parquet,70732568
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/part-00003-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22406-c000.snappy.parquet,part-00003-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22406-c000.snappy.parquet,70226369
dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet/part-00004-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22407-c000.snappy.parquet,part-00004-tid-3872868080952213493-d32b4468-0db9-431b-8ec8-1e97c3e7d738-22407-c000.snappy.parquet,70740225


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsParquet8
USING PARQUET 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-8p.parquet"
  )

In [0]:
%sql
SELECT count(`Incident_Number`)
FROM firecallsParquet8
WHERE Priority > 1

count(Incident_Number)
4401920


This file has 8 partitions rather than 1. Look at the speed improvement!

##Reading CSV
- `spark.read.csv(..)`
- There are a large number of options when reading CSV files including headers, column separator, escaping, etc.
- We can allow Spark to infer the schema at the cost of first reading in the entire file
- Large CSV files should always have a schema pre-defined

## Reading Parquet
- `spark.read.parquet(..)`
- Parquet files are the preferred file format for big-data
- It is a columnar file format
- It is a splittable file format
- It offers a lot of performance benefits over other formats including predicate push down
- Unlike CSV, the schema is read in, not inferred
- Reading the schema from Parquet's metadata can be extremely efficient

-sandbox

## Comparison
| Type    | <span style="white-space:nowrap">Inference Type</span> | <span style="white-space:nowrap">Inference Speed</span> | Reason                                          | <span style="white-space:nowrap">Should Supply Schema?</span> |
|---------|--------------------------------------------------------|---------------------------------------------------------|----------------------------------------------------|:--------------:|
| <b>CSV</b>     | <span style="white-space:nowrap">Full-Data-Read</span> | <span style="white-space:nowrap">Slow</span>            | <span style="white-space:nowrap">File size</span>  | Yes            |
| <b>Parquet</b> | <span style="white-space:nowrap">Metadata-Read</span>  | <span style="white-space:nowrap">Fast/Medium</span>     | <span style="white-space:nowrap">Number of Partitions</span> | No (most cases)             |
| <b>Tables</b>  | <span style="white-space:nowrap">n/a</span>            | <span style="white-space:nowrap">n/a</span>            | <span style="white-space:nowrap">Predefined</span> | n/a            |
| <b>JSON</b>    | <span style="white-space:nowrap">Full-Read-Data</span> | <span style="white-space:nowrap">Slow</span>            | <span style="white-space:nowrap">File size</span>  | Yes            |
| <b>Text</b>    | <span style="white-space:nowrap">Dictated</span>       | <span style="white-space:nowrap">Zero</span>            | <span style="white-space:nowrap">Only 1 Column</span>   | Never          |
| <b>JDBC</b>    | <span style="white-space:nowrap">DB-Read</span>        | <span style="white-space:nowrap">Fast</span>            | <span style="white-space:nowrap">DB Schema</span>  | No             |

## Reading Tables
- `spark.read.table(..)`
- The Databricks platform allows us to register a huge variety of data sources as tables via the Databricks UI
- Any `DataFrame` (from CSV, Parquet, whatever) can be registered as a temporary view
- Tables/Views can be loaded via the `DataFrameReader` to produce a `DataFrame`
- Tables/Views can be used directly in SQL statements

## Reading JSON
- `spark.read.json(..)`
- JSON represents complex data types unlike CSV's flat format
- Has many of the same limitations as CSV (needing to read the entire file to infer the schema)
- Like CSV has a lot of options allowing control on date formats, escaping, single vs. multiline JSON, etc.

## Reading Text
- `spark.read.text(..)`
- Reads one line of text as a single column named `value`
- Is the basis for more complex file formats such as fixed-width text files

## Reading JDBC
- `spark.read.jdbc(..)`
- Requires one database connection per partition
- Has the potential to overwhelm the database
- Requires specification of a stride to properly balance partitions

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>